In [1]:
!pip install git+https://github.com/roshan-research/hazm.git

  Cloning https://github.com/roshan-research/hazm.git to /tmp/pip-req-build-txt6xc0o
  Running command git clone --filter=blob:none --quiet https://github.com/roshan-research/hazm.git /tmp/pip-req-build-txt6xc0o
  Resolved https://github.com/roshan-research/hazm.git to commit 410e5f3e98bc267648b1f8c9e024c15c49c514b7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 28.0 MB/s eta 0:00:00
  Created wheel for hazm: filename=hazm-0.9.4-py3-none-any.whl size=882187 sha256=15782b5926e7ada6594bcfa7273e671192fb7d19d2f0d9f7bc5ae79c889912f6
  Stored in directory: /tmp/pip-e

In [1]:
!pip install transformers
!pip install -q scikit-plot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import re
from __future__ import unicode_literals
from hazm import Normalizer, word_tokenize
import plotly.express as px
import plotly.graph_objects as go
from sklearn.utils import class_weight
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,classification_report
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
# train_related_specific_df.Specific_comment.value_counts()

In [3]:
train_df = pd.read_csv('/content/train.csv',encoding ='utf-8')
test_df = pd.read_csv('/content/test.csv',encoding ='utf-8')
train_related_df = pd.read_csv('/content/train_related.csv',encoding ='utf-8')
test_related_df= pd.read_csv('/content/test_related.csv',encoding ='utf-8')
train_related_specific_df = pd.read_csv('/content/train_related_specific.csv',encoding ='utf-8')
test_related_specific_df = pd.read_csv('/content/test_related_specific.csv',encoding ='utf-8')


data_classes_= [-1,0,1]
d=dict(zip(data_classes_, range(0,3)))
train_related_df['InRelationship']=train_related_df['InRelationship'].map(d, na_action='ignore')
test_related_df['InRelationship']=test_related_df['InRelationship'].map(d, na_action='ignore')



train_df.drop(['Unnamed: 0'], axis=1,inplace=True)
test_df.drop(['Unnamed: 0'], axis=1,inplace=True)
train_related_df.drop(['Unnamed: 0'], axis=1,inplace=True)
test_related_df.drop(['Unnamed: 0'], axis=1,inplace=True)
train_related_specific_df.drop(['Unnamed: 0'], axis=1,inplace=True)
test_related_specific_df.drop(['Unnamed: 0'], axis=1,inplace=True)

In [4]:
from transformers import BertModel, AutoTokenizer

model = BertModel.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")


In [5]:
train_df['tokens_ids'] = train_df['Record_text'].apply(lambda t: tokenizer.encode(t, add_special_tokens=True))
test_df['tokens_ids'] = test_df['Record_text'].apply(lambda t: tokenizer.encode(t, add_special_tokens=True))

train_related_df['tokens_ids'] = train_related_df['Record_text'].apply(lambda t: tokenizer.encode(t, add_special_tokens=True))
test_related_df['tokens_ids'] = test_related_df['Record_text'].apply(lambda t: tokenizer.encode(t, add_special_tokens=True))

train_related_specific_df['tokens_ids'] = train_related_specific_df['Record_text'].apply(lambda t: tokenizer.encode(t, add_special_tokens=True))
test_related_specific_df['tokens_ids'] = test_related_specific_df['Record_text'].apply(lambda t: tokenizer.encode(t, add_special_tokens=True))

In [9]:
train_labels = train_df['Relevance'].values[:]
test_labels = test_df['Relevance'].values[:]


train_related_labels = train_related_df['General_comment'].values[:]
test_related_labels = test_related_df['General_comment'].values[:]

train_related_InRelationship_labels = train_related_df['InRelationship'].values[:]
test_related_InRelationship_labels = test_related_df['InRelationship'].values[:]


train_specific_labels = train_related_specific_df['Specific_comment'].values[:]
test_specific_labels = test_related_specific_df['Specific_comment'].values[:]

In [10]:
class_weights = class_weight.compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(train_labels),
                                        y = train_labels
                                    )


class_weights_specific = class_weight.compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(train_specific_labels),
                                        y = train_specific_labels
                                    )





class_weights_related = class_weight.compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(train_related_labels),
                                        y = train_related_labels
                                    )

weights = class_weight.compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(train_related_InRelationship_labels),
                                        y = train_related_InRelationship_labels
                                    )


class_weights = dict(zip(np.unique(train_labels), class_weights))
class_weights_related = dict(zip(np.unique(train_related_labels), class_weights_related))
weights = dict(zip(np.unique(train_related_InRelationship_labels), weights))
class_weights_specific = dict(zip(np.unique(train_specific_labels), class_weights_specific))


In [11]:
# Function to handle padding in each batch of data
def pad_collate(batch):
  (xx, yy) = zip(*batch)
  x_lens = [len(x) for x in xx]

  xx_pad = pad_sequence(xx, batch_first=True, padding_value=0)

  return xx_pad, yy

In [12]:
train_data = [[torch.tensor(train_df['tokens_ids'][i]), torch.tensor([train_labels[i]])] for i in range(len(train_df['tokens_ids']))]

train_related_data = [[torch.tensor(train_related_df['tokens_ids'][i]), torch.tensor([train_related_labels[i]])] for i in range(len(train_related_df['tokens_ids']))]

train_related_InRelationship_data = [[torch.tensor(train_related_df['tokens_ids'][i]), torch.tensor([train_related_InRelationship_labels[i]])] for i in range(len(train_related_df['tokens_ids']))]

train_specific_data =[[torch.tensor(train_related_specific_df['tokens_ids'][i]), torch.tensor([train_specific_labels[i]])] for i in range(len(train_related_specific_df['tokens_ids']))]



In [13]:
test_data = [[torch.tensor(test_df['tokens_ids'][i]), torch.tensor([test_labels[i]])] for i in range(len(test_df['tokens_ids']))]

test_related_data = [[torch.tensor(test_related_df['tokens_ids'][i]), torch.tensor([test_related_labels[i]])] for i in range(len(test_related_df['tokens_ids']))]

test_related_InRelationship_data = [[torch.tensor(test_related_df['tokens_ids'][i]), torch.tensor([test_related_InRelationship_labels[i]])] for i in range(len(test_related_df['tokens_ids']))]

test_specidic_data = [[torch.tensor(test_related_specific_df['tokens_ids'][i]), torch.tensor([test_specific_labels[i]])] for i in range(len(test_related_specific_df['tokens_ids']))]


In [16]:
train_set, val_set = torch.utils.data.random_split(train_data, [10771, 85])
train_related_set, val_related_set = torch.utils.data.random_split(train_related_data, [5455, 85])
train_related_InRelationship_set, val_related_InRelationship_set = torch.utils.data.random_split(train_related_InRelationship_data, [5455, 85])
train_specific_set, val_specific_set = torch.utils.data.random_split(train_specific_data, [2368, 85])

data_loader_Trianset = DataLoader(
    dataset=train_set,
    batch_size=512,
    shuffle=True,
    collate_fn=pad_collate
)

data_loader_Val = DataLoader(
    dataset=val_set,
    batch_size=512,
    shuffle=True,
    collate_fn=pad_collate
)

data_loader_Relatedset = DataLoader(
    dataset=train_related_set,
    batch_size=512,
    shuffle=True,
    collate_fn=pad_collate
)

data_loader_Valrelated = DataLoader(
    dataset=val_related_set,
    batch_size=512,
    shuffle=True,
    collate_fn=pad_collate
)


data_loader_InRelationship_Relatedset = DataLoader(
    dataset=train_related_InRelationship_set,
    batch_size=512,
    shuffle=True,
    collate_fn=pad_collate
)

data_loader_InRelationship_Valrelated = DataLoader(
    dataset=val_related_InRelationship_set,
    batch_size=512,
    shuffle=True,
    collate_fn=pad_collate
)



data_loader_Specific_set = DataLoader(
    dataset=train_specific_set,
    batch_size=512,
    shuffle=True,
    collate_fn=pad_collate
)


data_loader_Valspecific = DataLoader(
    dataset=val_specific_set,
    batch_size=512,
    shuffle=True,
    collate_fn=pad_collate
)




In [17]:
test_loader = DataLoader(
    dataset=test_data,
    batch_size=128,
    shuffle=True,
    collate_fn=pad_collate
)

test_loader_Related = DataLoader(
    dataset=test_related_data,
    batch_size=512,
    shuffle=True,
    collate_fn=pad_collate
)

test_loader_InRelationship_Related = DataLoader(
    dataset=test_related_InRelationship_data,
    batch_size=512,
    shuffle=True,
    collate_fn=pad_collate
)



test_loader_Specific = DataLoader(
    dataset=test_specidic_data,
    batch_size=512,
    shuffle=True,
    collate_fn=pad_collate
)

In [18]:
embedding_matrix = model.embeddings.word_embeddings.weight.detach().numpy()
print(embedding_matrix.shape)

(100000, 768)


In [19]:
class CNN_Model_TEXT(nn.Module):
    def __init__(self,max_features, embed_size, embedding_matrix, num_classes):
        super(CNN_Model_TEXT, self).__init__()
        filter_sizes = [3, 5, 7]
        num_filters = 36
        n_classes = num_classes
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = True
        self.convs1 = nn.ModuleList([nn.Conv2d(1, num_filters, (K, embed_size)) for K in filter_sizes])
        self.dropout = nn.Dropout(0.1)
        self.fc1 = nn.Linear(len(filter_sizes)*num_filters, n_classes)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.embedding(x)
        # print(x.size())
        x = x.unsqueeze(1)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]
        x = torch.cat(x, 1)
        x = self.dropout(x)
        logit = self.softmax(self.fc1(x))
        return logit

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [21]:
embed_size = embedding_matrix.shape[1]
vocab_size = embedding_matrix.shape[0]
num_classes = 2

model = CNN_Model_TEXT(vocab_size, embed_size, embedding_matrix,num_classes)
model.to(device)



criterion = nn.CrossEntropyLoss() # weighted negative log likelihood loss
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0005)

In [22]:
def training_loop(e, model,data_loader,valid_loader,criterion,optimizer):
  import time
  start_time = time.time()
  num_epochs = e
  train_loss = []
  validation_loss = []
  train_acc=[]
  val_acc = []
  for epoch in range(num_epochs):
    running_loss = 0
    running_acc=0
    # Training loop
    for idx, (sentence, label) in enumerate(iter(data_loader)):
      sentence, label = sentence.to(device),torch.tensor(list(label)).to(device)
      # Put the model into the training mode
      model.train()
      # Zero out any previously calculated gradients
      optimizer.zero_grad()
      # Feed forward
      outputs = model(sentence)
      # Calculate the batch loss and accuracy
      pred = torch.argmax(outputs, dim=1)
      running_acc += sum(pred==label).item() /len(label)
      loss = criterion(outputs, label)
      # Backward pass
      loss.backward()
      # Update the parameters in the optimize.
      optimizer.step()
      running_loss += loss.item()

      # print(loss.item())
    # validation
    model.eval()
    running_loss_valid = 0.0
    running_acc_valid = 0.0
    for jdx, (sentence, label) in enumerate(iter(valid_loader)):
      sentence, label = sentence.to(device),torch.tensor(list(label)).to(device)
      # Feed forward
      outputs = model(sentence)
      loss = criterion(outputs, label)
      # Calculate the batch loss and accuracy
      running_loss_valid += loss.item()
      pred = torch.argmax(outputs, dim=1)
      running_acc_valid += sum(pred==label).item()/len(label)
    # print training and validation info
    print(f'epoch: {epoch+1} -  train loss: {running_loss / (idx+1)} - validation loss: {running_loss_valid/(jdx+1)} - train acc = {running_acc/(idx+1)} - validation acc = {running_acc_valid/(jdx+1)}')
    train_loss.append(running_loss / (idx+1))
    validation_loss.append(running_loss_valid/(jdx+1))
    train_acc.append(running_acc/(idx+1))
    val_acc.append(running_acc_valid/(jdx+1))
    # saving model
    torch.save(model.cpu().state_dict(), 'final_model.pth')
    model.to(device)

  running_time = time.time()-start_time
  print(f'training Finished in {running_time} seconds')
  return train_loss,validation_loss,train_acc,val_acc

In [23]:
train_loss,validation_loss,train_acc,val_acc=training_loop(5,model,data_loader_Trianset,data_loader_Val,criterion,optimizer)

epoch: 1 -  train loss: 0.6842569654638117 - validation loss: 0.6647060513496399 - train acc = 0.5493047248803827 - validation acc = 0.5764705882352941
epoch: 2 -  train loss: 0.6144132099368356 - validation loss: 0.6003327965736389 - train acc = 0.6759868421052632 - validation acc = 0.6588235294117647
epoch: 3 -  train loss: 0.49651263518766925 - validation loss: 0.5224745869636536 - train acc = 0.7681388307416269 - validation acc = 0.7176470588235294
epoch: 4 -  train loss: 0.3979940875010057 - validation loss: 0.5413777232170105 - train acc = 0.8285875822368421 - validation acc = 0.7529411764705882
epoch: 5 -  train loss: 0.31844946267929947 - validation loss: 0.5759571194648743 - train acc = 0.8751822293660286 - validation acc = 0.7058823529411765
training Finished in 42.490407943725586 seconds


In [24]:
def Evaluate(model,test_loader):
  y_pred= []
  y_label=[]
  with torch.no_grad():
    for jdx, (sentence, label) in enumerate(iter(test_loader)):
      sentence, label = sentence.to(device),torch.tensor(list(label)).to(device)
      # Feed forward
      outputs = model(sentence)
      pred = torch.argmax(outputs, dim=1)
      for i in range(len(label)):
        y_pred.append(pred[i].item())
        y_label.append(label[i].item())
  print(classification_report(y_label, y_pred))
  print(f'''Accuracy: {round(accuracy_score(y_label, y_pred, normalize=True),6)}
  \nPrecision:{round(precision_score(y_label, y_pred, average='macro'),6)}
  \nRecall: {round(recall_score(y_label, y_pred, average='macro'),6)}
  \nF1 score: {round(f1_score(y_label, y_pred, average='macro'),6)}''')

In [25]:
Evaluate(model,test_loader)

              precision    recall  f1-score   support

           0       0.73      0.76      0.75      1156
           1       0.77      0.74      0.75      1227

    accuracy                           0.75      2383
   macro avg       0.75      0.75      0.75      2383
weighted avg       0.75      0.75      0.75      2383

Accuracy: 0.751154
  
Precision:0.751321
  
Recall: 0.751521
  
F1 score: 0.751129


In [26]:
embed_size_ = embedding_matrix.shape[1]
vocab_size_ = embedding_matrix.shape[0]
num_classes_ = 3

model_General_comment = CNN_Model_TEXT(vocab_size_, embed_size_, embedding_matrix,num_classes_)

model_General_comment.to(device)
criterion = nn.CrossEntropyLoss(weight=torch.tensor(list(class_weights_related.values())).float().to(device))
optimizer = optim.Adam(model_General_comment.parameters(), lr=0.0001, weight_decay=0.0005)

In [27]:
train_loss,validation_loss,train_acc,val_acc=training_loop(50,model_General_comment,data_loader_Relatedset,data_loader_Valrelated,criterion,optimizer)

epoch: 1 -  train loss: 1.0975790240547874 - validation loss: 1.0947635173797607 - train acc = 0.2430259921981004 - validation acc = 0.24705882352941178
epoch: 2 -  train loss: 1.0944092056967996 - validation loss: 1.092331051826477 - train acc = 0.4289587220149254 - validation acc = 0.4823529411764706
epoch: 3 -  train loss: 1.091740077192133 - validation loss: 1.089979648590088 - train acc = 0.4149147727272728 - validation acc = 0.4588235294117647
epoch: 4 -  train loss: 1.0890755653381348 - validation loss: 1.088133692741394 - train acc = 0.46321393741519673 - validation acc = 0.4588235294117647
epoch: 5 -  train loss: 1.086642330343073 - validation loss: 1.0856589078903198 - train acc = 0.4803966672320217 - validation acc = 0.47058823529411764
epoch: 6 -  train loss: 1.0841818072579124 - validation loss: 1.0833202600479126 - train acc = 0.47832958785617363 - validation acc = 0.47058823529411764
epoch: 7 -  train loss: 1.0810990116812966 - validation loss: 1.0812604427337646 - train

In [28]:
Evaluate(model_General_comment,test_loader_Related)

              precision    recall  f1-score   support

           0       0.49      0.52      0.51       269
           1       0.66      0.64      0.65       575
           2       0.46      0.45      0.46       381

    accuracy                           0.56      1225
   macro avg       0.54      0.54      0.54      1225
weighted avg       0.56      0.56      0.56      1225

Accuracy: 0.557551
  
Precision:0.535779
  
Recall: 0.53999
  
F1 score: 0.537638


In [32]:
embed_size_ = embedding_matrix.shape[1]
vocab_size_ = embedding_matrix.shape[0]
num_classes_ = 3

model_InRelationship = CNN_Model_TEXT(vocab_size_, embed_size_, embedding_matrix,num_classes_)

model_InRelationship.to(device)
criterion = nn.CrossEntropyLoss(weight=torch.tensor(list(weights.values())).float().to(device))
optimizer = optim.Adam(model_InRelationship.parameters(), lr=0.001, weight_decay=0.00005)

In [30]:
train_loss,validation_loss,train_acc,val_acc=training_loop(6,model_InRelationship,data_loader_InRelationship_Relatedset,data_loader_InRelationship_Valrelated,criterion,optimizer)

epoch: 1 -  train loss: 1.0866434249010952 - validation loss: 1.0665981769561768 - train acc = 0.5122662610244233 - validation acc = 0.49411764705882355
epoch: 2 -  train loss: 1.020786632191051 - validation loss: 1.0393606424331665 - train acc = 0.5861744190976933 - validation acc = 0.7058823529411765
epoch: 3 -  train loss: 0.8868551850318909 - validation loss: 0.9076188206672668 - train acc = 0.7495436524762552 - validation acc = 0.6588235294117647
epoch: 4 -  train loss: 0.7094096107916399 - validation loss: 0.8663300275802612 - train acc = 0.7954974771031207 - validation acc = 0.7176470588235294
epoch: 5 -  train loss: 0.5299023904583671 - validation loss: 1.028816819190979 - train acc = 0.8424933217435551 - validation acc = 0.7176470588235294
epoch: 6 -  train loss: 0.3738304999741641 - validation loss: 0.9931901693344116 - train acc = 0.8971506105834464 - validation acc = 0.7294117647058823
training Finished in 30.337883234024048 seconds


In [31]:
Evaluate(model_InRelationship,test_loader_InRelationship_Related)

              precision    recall  f1-score   support

           0       0.84      0.81      0.83       650
           1       0.20      0.29      0.24        51
           2       0.79      0.79      0.79       524

    accuracy                           0.78      1225
   macro avg       0.61      0.63      0.62      1225
weighted avg       0.79      0.78      0.79      1225

Accuracy: 0.781224
  
Precision:0.611225
  
Recall: 0.632291
  
F1 score: 0.618664


In [41]:
embed_size__ = embedding_matrix.shape[1]
vocab_size__ = embedding_matrix.shape[0]
num_classes__ = 3

model_Specific_comment = CNN_Model_TEXT(vocab_size__, embed_size__, embedding_matrix,num_classes__)

model_Specific_comment.to(device)


criterion = nn.CrossEntropyLoss(ignore_index=0,weight=torch.tensor(list(class_weights_specific.values())).float().to(device)) # weighted negative log likelihood loss
optimizer = optim.Adam(model_Specific_comment.parameters(),lr=0.0001, weight_decay=0.000005)

In [42]:
train_loss_,validation_loss_,train_acc_,val_acc_=training_loop(120,model_Specific_comment,data_loader_Specific_set,data_loader_Valspecific,criterion,optimizer)


epoch: 1 -  train loss: 1.0774925231933594 - validation loss: 1.0437296628952026 - train acc = 0.3509375 - validation acc = 0.38823529411764707
epoch: 2 -  train loss: 1.0274428129196167 - validation loss: 0.9999416470527649 - train acc = 0.34671875 - validation acc = 0.38823529411764707
epoch: 3 -  train loss: 0.9837229609489441 - validation loss: 0.9602578282356262 - train acc = 0.3565625 - validation acc = 0.4235294117647059
epoch: 4 -  train loss: 0.9435186147689819 - validation loss: 0.923900306224823 - train acc = 0.355 - validation acc = 0.3176470588235294
epoch: 5 -  train loss: 0.90645991563797 - validation loss: 0.8910467028617859 - train acc = 0.32671875 - validation acc = 0.3176470588235294
epoch: 6 -  train loss: 0.8741255283355713 - validation loss: 0.8621438145637512 - train acc = 0.3265625 - validation acc = 0.3176470588235294
epoch: 7 -  train loss: 0.8425963878631592 - validation loss: 0.8367784023284912 - train acc = 0.327734375 - validation acc = 0.3176470588235294


In [43]:
Evaluate(model_Specific_comment,test_loader_Specific)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       171
           1       0.42      0.61      0.50       181
           2       0.43      0.62      0.51       195

    accuracy                           0.42       547
   macro avg       0.28      0.41      0.33       547
weighted avg       0.29      0.42      0.34       547

Accuracy: 0.422303
  
Precision:0.281436
  
Recall: 0.409416
  
F1 score: 0.333572


In [44]:
class BiLSTM(nn.Module):

    def __init__(self,max_features, embed_size, embedding_matrix, num_classes, hidden_size):
        super(BiLSTM, self).__init__()
        self.hidden_size = hidden_size
        drp = 0.5
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = True
        self.lstm = nn.LSTM(embed_size, self.hidden_size, bidirectional=True, batch_first=True)
        self.linear = nn.Linear(self.hidden_size*4 , 32)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drp)
        self.out = nn.Linear(32, num_classes)
        self.softmax = nn.LogSoftmax(dim=1)


    def forward(self, x):
        h_embedding = self.embedding(x)
        h_embedding = torch.squeeze(torch.unsqueeze(h_embedding, 0))

        h_lstm, _ = self.lstm(h_embedding)
        avg_pool = torch.mean(h_lstm, 1)
        max_pool, _ = torch.max(h_lstm, 1)
        conc = torch.cat(( avg_pool, max_pool), 1)
        conc = self.relu(self.linear(conc))
        conc = self.dropout(conc)
        out = self.out(conc)
        out = self.softmax(out)
        return out

In [45]:
def training_loop_(e, model,data_loader,valid_loader,criterion,optimizer):
# number of epochs to train the model
    import time
    start_time = time.time()
    n_epochs = e
   # test_loss_min = np.Inf # track change in validation loss
    train_acc=[]
    val_acc = []
    for epoch in range(n_epochs):

        # keep track of training and validation loss
        train_loss = 0.0
        test_loss = 0.0

        ###################
        # train the model #
        ###################
        model.train()
        running_acc=0
        for i, (data, target) in enumerate(data_loader):
            # move tensors to GPU if CUDA is available
            target = torch.tensor([[t] for t in target])
            data, target = data.to(device),torch.tensor(list(target)).to(device)
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            # target = target.Long()
            loss = criterion(output, target)
            y_pred = torch.round(torch.sigmoid(output))
            output =  torch.argmax(output, dim=1).float()

            running_acc += torch.round((((output == target).sum().float() /len(target))) * 100)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_loss += loss.item()

        ######################
        # validate the model #
        ######################
        model.eval()
        running_acc_valid = 0.0
        for j,(data, target) in enumerate(valid_loader):
            # target = target.unsqueeze(1)
            target = torch.tensor([[t] for t in target])
            data, target = data.to(device),torch.tensor(list(target)).to(device)
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            output =  torch.argmax(output, dim=1).float()
            running_acc_valid += torch.round((((output == target).sum().float() /len(target))) * 100)
            #break
            # update average validation loss
            test_loss += loss.item()

        # calculate average losses
        print(f'epoch: {epoch+1} -  train loss: {train_loss / (i+1)} - validation loss: {test_loss/(j+1)} - train acc = {running_acc/(i+1)} - validation acc = {running_acc_valid/(j+1)}')

        train_loss = train_loss/len(data_loader)
        test_loss = test_loss/len(valid_loader)
        train_acc.append(running_acc/(i+1))
        val_acc.append(running_acc_valid/(j+1))
        torch.save(model.cpu().state_dict(), 'final_model.pth')
        model.to(device)

    running_time = time.time()-start_time
    print(f'training Finished in {running_time} seconds')
    return train_loss,test_loss,train_acc,val_acc

In [46]:
embed_size = embedding_matrix.shape[1]
vocab_size = embedding_matrix.shape[0]
num_classes = 2

model_Relevance_Bilstm = BiLSTM(vocab_size, embed_size, embedding_matrix,num_classes,10)
model_Relevance_Bilstm.to(device)
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.Adam(model_Relevance_Bilstm.parameters(), lr=0.001, weight_decay=0.0005)


In [47]:
train_loss,validation_loss,train_acc,val_acc=training_loop_(8,model_Relevance_Bilstm,data_loader_Trianset,data_loader_Val,criterion,optimizer)

epoch: 1 -  train loss: 0.6902062161402269 - validation loss: 0.6843487024307251 - train acc = 52.54545593261719 - validation acc = 55.0
epoch: 2 -  train loss: 0.6771092739972201 - validation loss: 0.6669309139251709 - train acc = 59.22727584838867 - validation acc = 66.0
epoch: 3 -  train loss: 0.643615332516757 - validation loss: 0.6338107585906982 - train acc = 66.90909576416016 - validation acc = 68.0
epoch: 4 -  train loss: 0.5795175446705385 - validation loss: 0.5928585529327393 - train acc = 72.45454406738281 - validation acc = 68.0
epoch: 5 -  train loss: 0.5142255574464798 - validation loss: 0.5624984502792358 - train acc = 76.7272720336914 - validation acc = 72.0
epoch: 6 -  train loss: 0.44510172036561096 - validation loss: 0.5840638875961304 - train acc = 81.18182373046875 - validation acc = 72.0
epoch: 7 -  train loss: 0.36472083357247437 - validation loss: 0.5547714233398438 - train acc = 85.5 - validation acc = 76.0
epoch: 8 -  train loss: 0.30904829095710407 - validati

In [48]:
Evaluate(model_Relevance_Bilstm,test_loader)

              precision    recall  f1-score   support

           0       0.76      0.67      0.71      1156
           1       0.72      0.80      0.76      1227

    accuracy                           0.74      2383
   macro avg       0.74      0.73      0.73      2383
weighted avg       0.74      0.74      0.73      2383

Accuracy: 0.736047
  
Precision:0.738451
  
Recall: 0.734223
  
F1 score: 0.734212


In [49]:
embed_size_ = embedding_matrix.shape[1]
vocab_size_ = embedding_matrix.shape[0]
num_classes_ = 3

model_General_comment_Bilstm = BiLSTM(vocab_size_, embed_size_, embedding_matrix,num_classes_,10)


model_General_comment_Bilstm.to(device)
criterion = nn.CrossEntropyLoss(weight=torch.tensor(list(class_weights_related.values())).float().to(device))

# specify optimizer
optimizer = optim.Adam(model_General_comment_Bilstm.parameters(), lr=0.001, weight_decay=0.000005)

In [50]:
train_loss,validation_loss,train_acc,val_acc=training_loop_(8,model_General_comment_Bilstm,data_loader_Relatedset,data_loader_Valrelated,criterion,optimizer)

epoch: 1 -  train loss: 1.1001287373629483 - validation loss: 1.0903270244598389 - train acc = 43.45454788208008 - validation acc = 48.0
epoch: 2 -  train loss: 1.085147044875405 - validation loss: 1.076188087463379 - train acc = 44.181819915771484 - validation acc = 52.0
epoch: 3 -  train loss: 1.0558179183439775 - validation loss: 1.054761528968811 - train acc = 51.272727966308594 - validation acc = 49.0
epoch: 4 -  train loss: 1.0050976493141868 - validation loss: 1.0243980884552002 - train acc = 59.6363639831543 - validation acc = 54.0
epoch: 5 -  train loss: 0.9175972450863231 - validation loss: 0.9913379549980164 - train acc = 66.63636779785156 - validation acc = 55.0
epoch: 6 -  train loss: 0.7861078002236106 - validation loss: 0.9636892676353455 - train acc = 75.0 - validation acc = 55.0
epoch: 7 -  train loss: 0.6408720449967817 - validation loss: 0.9985739588737488 - train acc = 80.18182373046875 - validation acc = 59.0
epoch: 8 -  train loss: 0.49201017347249115 - validation

In [51]:
Evaluate(model_General_comment_Bilstm,test_loader_Related)

              precision    recall  f1-score   support

           0       0.42      0.48      0.45       269
           1       0.67      0.61      0.64       575
           2       0.45      0.47      0.46       381

    accuracy                           0.54      1225
   macro avg       0.51      0.52      0.52      1225
weighted avg       0.55      0.54      0.54      1225

Accuracy: 0.537959
  
Precision:0.514549
  
Recall: 0.51964
  
F1 score: 0.515887


In [52]:
embed_size_ = embedding_matrix.shape[1]
vocab_size_ = embedding_matrix.shape[0]
num_classes_ = 3

model_InRelationship_Bilstm = BiLSTM(vocab_size_, embed_size_, embedding_matrix,num_classes_,10)


model_InRelationship_Bilstm.to(device)

criterion = nn.CrossEntropyLoss(weight=torch.tensor(list(weights.values())).float().to(device))

# specify optimizer
optimizer = optim.Adam(model_InRelationship_Bilstm.parameters(),lr=0.0001, weight_decay=0.000005)

In [53]:
train_loss,validation_loss,train_acc,val_acc=training_loop_(50,model_InRelationship_Bilstm,data_loader_InRelationship_Relatedset,data_loader_InRelationship_Valrelated,criterion,optimizer)

epoch: 1 -  train loss: 1.102971781383861 - validation loss: 1.1079858541488647 - train acc = 53.36363983154297 - validation acc = 51.0
epoch: 2 -  train loss: 1.1008451093326916 - validation loss: 1.107722520828247 - train acc = 53.181819915771484 - validation acc = 51.0
epoch: 3 -  train loss: 1.098419644615867 - validation loss: 1.1066498756408691 - train acc = 53.272727966308594 - validation acc = 51.0
epoch: 4 -  train loss: 1.0970791469920764 - validation loss: 1.1057851314544678 - train acc = 52.90909194946289 - validation acc = 51.0
epoch: 5 -  train loss: 1.0960886911912398 - validation loss: 1.1049580574035645 - train acc = 52.90909194946289 - validation acc = 51.0
epoch: 6 -  train loss: 1.095172806219621 - validation loss: 1.1037323474884033 - train acc = 52.36363983154297 - validation acc = 51.0
epoch: 7 -  train loss: 1.0918497280641035 - validation loss: 1.102733850479126 - train acc = 53.090911865234375 - validation acc = 51.0
epoch: 8 -  train loss: 1.0892722823403098 

In [54]:
Evaluate(model_InRelationship_Bilstm,test_loader_InRelationship_Related)

              precision    recall  f1-score   support

           0       0.81      0.77      0.79       650
           1       0.13      0.12      0.12        51
           2       0.70      0.76      0.73       524

    accuracy                           0.74      1225
   macro avg       0.55      0.55      0.55      1225
weighted avg       0.74      0.74      0.73      1225

Accuracy: 0.73551
  
Precision:0.548487
  
Recall: 0.547022
  
F1 score: 0.547099


In [55]:
embed_size__ = embedding_matrix.shape[1]
vocab_size__ = embedding_matrix.shape[0]
num_classes__ = 3


model_Specific_comment_Bilstm  = BiLSTM(vocab_size__, embed_size__, embedding_matrix,num_classes__,10)


model_Specific_comment_Bilstm.to(device)


criterion = nn.CrossEntropyLoss(weight=torch.tensor(list(class_weights_specific.values())).float().to(device))
optimizer = optim.Adam(model_Specific_comment_Bilstm.parameters(), lr=0.0001, weight_decay=0.000005)

In [56]:
train_loss_,validation_loss_,train_acc_,val_acc_=training_loop(80,model_Specific_comment_Bilstm,data_loader_Specific_set,data_loader_Valspecific,criterion,optimizer)


epoch: 1 -  train loss: 1.0992408514022827 - validation loss: 1.0981965065002441 - train acc = 0.33546875 - validation acc = 0.3176470588235294
epoch: 2 -  train loss: 1.098192572593689 - validation loss: 1.0981234312057495 - train acc = 0.34109375000000003 - validation acc = 0.3176470588235294
epoch: 3 -  train loss: 1.0982757568359376 - validation loss: 1.0980936288833618 - train acc = 0.3434375 - validation acc = 0.3176470588235294
epoch: 4 -  train loss: 1.0963871240615846 - validation loss: 1.0979899168014526 - train acc = 0.34921875 - validation acc = 0.32941176470588235
epoch: 5 -  train loss: 1.0950767517089843 - validation loss: 1.0978015661239624 - train acc = 0.372109375 - validation acc = 0.36470588235294116
epoch: 6 -  train loss: 1.095755124092102 - validation loss: 1.0977022647857666 - train acc = 0.3603125 - validation acc = 0.3764705882352941
epoch: 7 -  train loss: 1.0957040309906005 - validation loss: 1.0976353883743286 - train acc = 0.367421875 - validation acc = 0.

In [57]:
Evaluate(model_Specific_comment_Bilstm,test_loader_Specific)

              precision    recall  f1-score   support

           0       0.51      0.46      0.48       171
           1       0.46      0.46      0.46       181
           2       0.48      0.51      0.49       195

    accuracy                           0.48       547
   macro avg       0.48      0.48      0.48       547
weighted avg       0.48      0.48      0.48       547

Accuracy: 0.478976
  
Precision:0.480567
  
Recall: 0.477683
  
F1 score: 0.478455
